In [1]:
import os 
import time
import pandas as pd


import keras 
import tensorflow as tf
from keras import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import BatchNormalization
from keras.layers import TimeDistributed
from keras.models import Model
from keras.layers import Conv2D
from keras.layers import LeakyReLU
from keras.layers import Activation
from keras.layers import Flatten
from keras.layers import Reshape
from keras.layers import Input
from keras import layers
from keras.layers import Conv2DTranspose

from keras import backend as K
from keras.layers import TimeDistributed
import numpy as np
import librosa
import matplotlib.pyplot as plt
import librosa.display

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import f1_score


2023-06-08 18:25:45.162798: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-08 18:25:45.164111: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-08 18:25:45.188667: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-08 18:25:45.189099: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-08 18:25:45.556564: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fin

# Motor vibrations bearing defects

## Data imports

In [2]:
train_data_2 = pd.read_csv("dataset/train.csv")
test_data_2 = pd.read_csv("dataset/test.csv")

## Model setup( CNN AutoEncoder ) 

In [3]:
class ConvAutoencoder:
    @staticmethod
    def build(width, height, depth, filters=(32, 64), latentDim=100):
        # initialize the input shape to be "channels last" along with
        # the channels dimension itself
        # channels dimension itself
        inputShape = (height, width, depth)
        chanDim = -1
        # define the input to the encoder
        inputs = Input(shape=inputShape)
        x = inputs
        # loop over the number of filters
        for f in filters:
        # apply a CONV => RELU => BN operation
            x = Conv2D(f, (3, 3), strides=2, padding="same")(x)
            x = LeakyReLU(alpha=0.2)(x)
            x = BatchNormalization(axis=chanDim)(x)
        # flatten the network and then construct our latent vector
        volumeSize = K.int_shape(x)
        x = Flatten()(x)
        latent = Dense(latentDim)(x)
        # build the encoder model
        encoder = Model(inputs, latent, name="encoder")
        # start building the decoder model which will accept the
        # output of the encoder as its inputs
        latentInputs = Input(shape=(latentDim,))
        x = Dense(np.prod(volumeSize[1:]))(latentInputs)
        x = Reshape((volumeSize[1], volumeSize[2], volumeSize[3]))(x)
        # loop over our number of filters again, but this time in
        # reverse order
        for f in filters[::-1]:
            # apply a CONV_TRANSPOSE => RELU => BN operation
            x = Conv2DTranspose(f, (3, 3), strides=2,
                padding="same")(x)
            x = LeakyReLU(alpha=0.2)(x)
            x = BatchNormalization(axis=chanDim)(x)
        # apply a single CONV_TRANSPOSE layer used to recover the
        # original depth of the image
        x = Conv2DTranspose(depth, (3, 3), padding="same")(x)
        outputs = Activation("sigmoid")(x)
        # build the decoder model
        decoder = Model(latentInputs, outputs, name="decoder")
        # our autoencoder is the encoder + decoder
        autoencoder = Model(inputs, decoder(encoder(inputs)),
        name="autoencoder")
        optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
        autoencoder.compile(loss="mse", optimizer=optimizer)
        return autoencoder

## Data formating

In [ ]:

def build_spectrograms(data,window,saving_folder,step,label_col = 'Class'):
    i = 0
    range_df = i*step
    while range_df+window < len(data):
        range_df = i*step
        y = data[range_df:range_df+window].value.values
        img_class = 1 if data[range_df:range_df+window][label_col].sum() > 0.3*window else  0
        S_left = librosa.stft(y, center=True, n_fft=int(window/2))
        fig, ax = plt.subplots()
        img = librosa.display.specshow(librosa.amplitude_to_db(S_left,
                                                               ref=np.max),
                                       y_axis='log', x_axis='time', ax=ax)
        ax.set_title('Power spectrogram')
        fig.colorbar(img, ax=ax, format="%+2.0f dB")
        plt.savefig(f"{saving_folder}/class_{img_class}/{range_df}_{range_df+window-1}.png")
        print(f"{saving_folder}/class_{img_class}/{range_df}_{range_df+window-1}.png")
        i+=1


In [ ]:
# build_spectrograms(train_data_2,500,"Spectrograms/Train",200)
# build_spectrograms(test_data_2,500,"Spectrograms/Test",200)

## Spectrograms loading

In [4]:
# Custom loading of training data
train_folder = "dataset/Spectrograms/Train"
train_images = []
ano_images = [] 
class_data = ["class_0","class_1"]
for label in class_data:
    for i,img in enumerate(os.listdir(f"{train_folder}/{label}/")):
        if 'png' in img:
            image = tf.keras.utils.load_img(f"{train_folder}/{label}/{img}",color_mode = "grayscale",target_size=(100, 100))
            input_arr = tf.keras.utils.img_to_array(image)/255.0
            if label == "class_0":    
                train_images.append(input_arr)
            else:
                ano_images.append(input_arr)


In [5]:
X_train0_CAE = np.array(train_images) # Training data without anomalies
X_train1_CAE = np.array(ano_images) # Training data with anomalies

In [6]:
# Custom loading of test data
test_folder = "dataset/Spectrograms/Test"
test_images0 = []
test_images1 = [] 
class_data = ["class_0","class_1"]
for label in class_data:
    for i,img in enumerate(os.listdir(f"{test_folder}/{label}/")):
        if 'png' in img:
            image = tf.keras.utils.load_img(f"{test_folder}/{label}/{img}",color_mode = "grayscale",target_size=(100, 100))
            input_arr = tf.keras.utils.img_to_array(image)/255.0
            if label == "class_0":    
                test_images0.append(input_arr)
            else:
                test_images1.append(input_arr)

In [7]:
X_test0CAE = np.array(test_images0) # Test data without anomalies
X_test1CAE = np.array(test_images1) # Test data with anomalies

## Model learning

In [8]:
CAE = ConvAutoencoder.build(100, 100, 1) # Model architecture

In [9]:
start = time.time()
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)
CAE.fit(X_train0_CAE,X_train0_CAE, epochs=100, verbose = True, callbacks= [callback])
print(f"model trained in {(time.time()-start)/60} minutes")

Epoch 1/100
36/36 [==============================] - 4s 93ms/step - loss: 0.0338
Epoch 2/100
36/36 [==============================] - 3s 93ms/step - loss: 0.0099
Epoch 3/100
36/36 [==============================] - 3s 92ms/step - loss: 0.0075
Epoch 4/100
36/36 [==============================] - 3s 92ms/step - loss: 0.0043
Epoch 5/100
36/36 [==============================] - 3s 92ms/step - loss: 0.0037
Epoch 6/100
36/36 [==============================] - 3s 92ms/step - loss: 0.0035
Epoch 7/100
36/36 [==============================] - 4s 98ms/step - loss: 0.0033
Epoch 8/100
36/36 [==============================] - 4s 107ms/step - loss: 0.0030
Epoch 9/100
36/36 [==============================] - 4s 108ms/step - loss: 0.0028
Epoch 10/100
36/36 [==============================] - 4s 107ms/step - loss: 0.0026
Epoch 11/100
36/36 [==============================] - 4s 107ms/step - loss: 0.0024
Epoch 12/100
36/36 [==============================] - 4s 107ms/step - loss: 0.0023
Epoch 13/100
36/36 [

## Prediction

In [10]:
predictions0CAE = CAE.predict(X_train0_CAE) # Prediction on data without anomalies
predictions1CAE = CAE.predict(X_train1_CAE) # Prediction on data with anomalies

9/9 [==============================] - 0s 33ms/step


In [11]:
r_e_CAE = np.sum(np.sum((X_train0_CAE-predictions0CAE) ** 2, axis=1),axis = 1) # Computing reconstruction error of training data without anomalies
r_e_anos_CAE = np.sum(np.sum((X_train1_CAE-predictions1CAE) ** 2, axis=1),axis = 1) # Computing reconstruction error of training data with anomalies
threshold_t_CAE = np.quantile(r_e_CAE,0.95) # Setting reconstruction error threshold

In [12]:
prediction_test0CAE = CAE.predict(X_test0CAE)
prediction_test1CAE = CAE.predict(X_test1CAE)

8/8 [==============================] - 0s 32ms/step


In [13]:
r_e_test0CAE = np.sum(np.sum((X_test0CAE-prediction_test0CAE) ** 2, axis=1),axis=1)
r_e_test1CAE = np.sum(np.sum((X_test1CAE-prediction_test1CAE) ** 2, axis=1),axis=1)

In [14]:
decision0CAE = -r_e_test0CAE + threshold_t_CAE
decision1CAE = -r_e_test1CAE + threshold_t_CAE
df0CAE = pd.DataFrame(decision0CAE, columns=['v'])
df1CAE = pd.DataFrame(decision1CAE, columns=['v'])

In [15]:
# Computing f1 score of minority class
recall = len(df1CAE[df1CAE.v<0])/len(df1CAE)
precision = len(df1CAE[df1CAE.v<0])/(len(df1CAE[df1CAE.v<0])+len(df0CAE[df0CAE.v<0]))
f1 = 2*precision*recall/(precision+recall)

In [16]:
# Computing f1 score of majority class
recall_maj = len(df0CAE[df0CAE.v>0])/len(df0CAE)
precision_maj = len(df0CAE[df0CAE.v>0])/(len(df1CAE[df1CAE.v>0])+len(df0CAE[df0CAE.v>0]))
f1_maj = 2*precision_maj*recall_maj/(precision_maj+recall_maj)

### Test score

In [17]:
print(f"Test scores:")
print(f"minority class F1 score {f1}")
print(f"majority class F1 score {f1_maj}")
print(f"average F1 score {(f1+f1_maj)/2}")

Test scores:
minority class F1 score 0.7096774193548387
majority class F1 score 0.9293193717277486
average F1 score 0.8194983955412937
